In [1]:
pip install git+https://github.com/huggingface/trl.git


  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-xh78sezk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-xh78sezk
  Resolved https://github.com/huggingface/trl.git to commit 05bc43e960396581e458195b8388efe6b82cae1f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
 

In [2]:
import trl
print(trl.__version__)

0.18.0.dev0


In [3]:
import os
import torch
import wandb
import gc
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from peft import LoraConfig, get_peft_model
from trl import DPOConfig, DPOTrainer  # <-- Thêm DPOConfig
import random
import numpy as np

In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [5]:
wandb.init(project="vietnamese-dpo-training", name="dpo-training-run")

# Configuration parameters
model_name = "hoa12356/Llama-3.2-1B-Instruct-Chat-sft-hoa"
dataset_name = "thainq107/Vi-Alpaca-Preference"
max_samples = 6000
output_dir = "./dpo_results"
os.makedirs(output_dir, exist_ok=True)

# Load dataset
print(f"Đang tải dataset từ {dataset_name}...")
dataset = load_dataset(dataset_name)
print(f"Đã tải dataset: {dataset}")

# Giới hạn số lượng mẫu
if "train" in dataset and len(dataset["train"]) > max_samples:
    print(f"Giới hạn tập huấn luyện xuống {max_samples} mẫu")
    dataset["train"] = dataset["train"].select(range(max_samples))

# Kiểm tra cấu trúc dữ liệu
required_columns = ["question", "chosen", "rejected"]
for col in required_columns:
    if col not in dataset["train"].column_names:
        raise ValueError(f"Thiếu cột bắt buộc: {col}")

# Load tokenizer
print(f"Đang tải tokenizer từ {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lhoa517 (lhoa517-phenikaa-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Đang tải dataset từ thainq107/Vi-Alpaca-Preference...


...

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Đã tải dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'chosen', 'rejected'],
        num_rows: 65017
    })
    test: Dataset({
        features: ['id', 'question', 'chosen', 'rejected'],
        num_rows: 2000
    })
})
Giới hạn tập huấn luyện xuống 6000 mẫu
Đang tải tokenizer từ hoa12356/Llama-3.2-1B-Instruct-Chat-sft-hoa...


tokenizer_config.json:   0%|          | 0.00/54.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [6]:
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Load model
print(f"Đang tải mô hình từ {model_name}...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
)
model.train()
model.config.use_cache = False

Đang tải mô hình từ hoa12356/Llama-3.2-1B-Instruct-Chat-sft-hoa...


config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [7]:
print("Áp dụng LoRA cho mô hình...")
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Verify trainable params
trainable_params = [p for p in model.parameters() if p.requires_grad]
print(f"Number of trainable parameters: {len(trainable_params)}")

Áp dụng LoRA cho mô hình...
trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750
Number of trainable parameters: 128


In [ ]:
def format_for_dpo(ex):
    return {"prompt": ex["question"], "chosen": ex["chosen"], "rejected": ex["rejected"]}

dpo_dataset = {}
for split in dataset:
    dpo_dataset[split] = dataset[split].map(format_for_dpo)
    print(f"Split {split}: {len(dpo_dataset[split])} examples")
def reward_num_unique_chars(completions, **kwargs):
    return [len(set(c)) for c in completions]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Split train: 6000 examples


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Split test: 2000 examples


In [11]:
from trl import DPOConfig, DPOTrainer

dpo_config = DPOConfig(
    output_dir=output_dir,
    run_name="dpo-training-run",
    # bỏ num_train_epochs hoặc để nhỏ hơn, không quá quan trọng khi dùng max_steps
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
    report_to="wandb",
    remove_unused_columns=False,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    # <-- thêm dòng này
    max_steps=300,
)
print("Khởi tạo DPOTrainer...")
dpo_trainer = DPOTrainer(
    model=model,
    args=dpo_config,
    train_dataset=dpo_dataset["train"],
    eval_dataset=dpo_dataset.get("validation", None),
    processing_class=tokenizer,
)


Khởi tạo DPOTrainer...


Applying chat template to train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [12]:

print("Bắt đầu huấn luyện DPO...")
dpo_trainer.train()
print("Huấn luyện hoàn tất!")

dpo_trainer.save_model(f"{output_dir}/final_model")
model.save_pretrained(f"{output_dir}/adapter_model")
wandb.finish()
print("Quá trình đã hoàn tất!")

Bắt đầu huấn luyện DPO...


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.161900
20,0.113400
30,0.124600
40,0.188600
50,0.118400
60,0.130100
70,0.123400
80,0.063800
90,0.047700
100,0.184500


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Huấn luyện hoàn tất!


train/epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██▁▁▂▂▃▃▃▄▄▄▅▆▆▆▆▇▇▇██
train/global_step,▁▁▂▂▂▃▃▄▄▄▅▅▆▆▆▇██▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▆▇▇███
train/grad_norm,▃▃ ▃▃▄▃▂▂▃▆▄▄▄▁▅▁▂▃▇█▁▁▂▃▂▂▇▂▄▃▁▃▃▅▁▃▄▂▂
train/learning_rate,▁▂▃▃▄▆▆████████████████▂███▇▇▇▆▆▅▅▅▃▃▂▁▁
train/logits/chosen,▄▃▃▄▃▃▃▃▂▂▁▂▃▃▂▄▅▄▃▃▃▃▂▂▄▅▅▇▅▄▄▄▆▅▆▇▇█▇█
train/logits/rejected,▃▃▃▃▃▂▃▂▁▂▂▂▃▃▅▄▄▃▂▂▂▂▁▂▂▄▄▆▄▃▃▃▄▄▄▅▆▇██
train/logps/chosen,▇▄█▇▄▅▇▅▅▄▅▇▃▇▄▄▆▃▅█▄▆▂▆▃▂▅█▄▅▄▆▄▂▁▃▂▅▂▂
train/logps/rejected,█▇█▇█▇▇▇▇▇▅▅▄▃▅▅▅▅▅▅▃▅▄▄▆▃▄▄▅▄▂▄▂▃▂▁▁▂▁▂
train/loss,████▇▇▆▅▄▄▃▃▃▃▃▃▃▂▂▃▂▂▃▂▂▁▂▂▂▁▂▁▁▂▁▁▁▁▁▂
train/rewards/accuracies,▁▆▆▆▆▇▇▇▇▇▇▇▇▇█▇█▇█▇▇█▇▇████████████████
train/rewards/chosen,███████▇▇▇▆▆▆▆▅▂▅▆▅▆▆▅▅▆▄▆▆▄▅▅▅▅▅▂▃▂▂▁▂▂


Quá trình đã hoàn tất!


In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [19]:
from huggingface_hub import create_repo

repo_name = "Llama-3.2-1B-Instruct-DPO-LoRA"
create_repo(repo_name, private=False)  # hoặc private=True nếu muốn riêng tư
from huggingface_hub import upload_folder

upload_folder(
    folder_path="./dpo_results/final_model",
    repo_id="hoa12356/Llama-3.2-1B-Instruct-DPO-LoRA",
    repo_type="model"
)


training_args.bin:   0%|          | 0.00/6.20k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/hoa12356/Llama-3.2-1B-Instruct-DPO-LoRA/commit/dd59e0cba037861f7024a8c3d6da1f02b5501f7c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='dd59e0cba037861f7024a8c3d6da1f02b5501f7c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hoa12356/Llama-3.2-1B-Instruct-DPO-LoRA', endpoint='https://huggingface.co', repo_type='model', repo_id='hoa12356/Llama-3.2-1B-Instruct-DPO-LoRA'), pr_revision=None, pr_num=None)

In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel, PeftConfig
import torch

# Tên model adapter LoRA trên Hugging Face
adapter_id = "hoa12356/Llama-3.2-1B-Instruct-DPO-LoRA"

# Tự động lấy base model từ adapter
config = PeftConfig.from_pretrained(adapter_id)
base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
# Load adapter
model = PeftModel.from_pretrained(base_model, adapter_id)
model.eval()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Dùng pipeline để dễ test
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
)

# Prompt thử nghiệm
prompt = "Viết một đoạn giới thiệu về Trí tuệ nhân tạo bằng tiếng Việt."

# Generate
output = pipe(prompt)
print(output[0]["generated_text"])


adapter_config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

Viết một đoạn giới thiệu về Trí tuệ nhân tạo bằng tiếng Việt. 

Giới thiệu về Trí tuệ nhân tạo - một lĩnh vực mở và thú vị, mang đến cho chúng ta những khả năng giải trí chưa từng có, những cơ hội mới và những điều chưa biết. Viết một đoạn giới thiệu về Trí tuệ nhân tạo sẽ mang lại cho bạn sự hấp dẫn sâu sắc hơn về thế giới này. Trí tuệ nhân tạo là một lĩnh vực mở và thú vị, mang đến cho chúng ta những khả năng giải trí chưa từng thấy, những cơ hội mới và những điều chưa biết. Với sự hỗ trợ của các thuật toán tiên tiến, Trí tuệ nhân tạo đang trở nên ngày càng quan trọng trong cuộc sống của chúng ta. Chúng tôi mời bạn đón nhận đoạn giới thiệu về Trí tuệ nhân tạo, nơi khám phá những bí mật tò tò, những dự án tiềm năng và những tương lai đầy hứa hẹn mà nó mang lại. Hãy cùng nhau khám phá thế giới Trí tuệ nhân tạo!
